In [15]:
%matplotlib inline
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.layers.slice import Slice
from CMS_SURF_2016.layers.lorentz import Lorentz
from keras.utils.visualize_util import plot
from IPython.display import Image, display
from CMS_SURF_2016.utils.colors import colors_contrasting
from CMS_SURF_2016.utils.analysistools import *
from CMS_SURF_2016.utils.plot import *
from CMS_SURF_2016.utils.metrics import *
import numpy as np
from CMS_SURF_2016.utils.plot import plot_history, print_accuracy_m
archive_dir = "/data/shared/Delphes/keras_archive/"

#def sortTrialsOn(lst):
    

In [16]:
trials = get_trials_by_name("LSTM", archive_dir)
#trials = findWithMetrics(trials, {"useObjTypeColumn" : True})
trials = [t for t in trials if t.is_complete()]
for trial in trials:
    trial.summary(showTraining=False,showValidation=False, showFit=False, showCompilation=False)
print("TotalNumber of Trials:", len(trials)) 

--------------------------------------------------
TRIAL SUMMARY (1d645dbd33ca1a4ec695e8cc49d4a4b6f848efda)
    Record_Info:
        name = [u'LSTM'], elapse_time = 4:14:32
        val_acc = 0.7873
        num_train = 150000
        activation = tanh, depth = 1, dropout = 0.0, labels = [u'qcd', u'ttbar'], lstm_dropout = 0.0, num_val = 150000, output_activation = softmax, patience = 8, query = None, sort_on = Eta, useObjTypeColumn = True
--------------------------------------------------
--------------------------------------------------
TRIAL SUMMARY (0131079e4a5cbf5fba3ddcfbf3182f8e10c4fb17)
    Record_Info:
        name = [u'LSTM'], elapse_time = 6:41:01
        val_acc = 0.8388
        num_train = 225000
        activation = tanh, depth = 1, dropout = 0.0, labels = [u'ttbar', u'wjet', u'qcd'], lstm_dropout = 0.0, num_val = 225000, output_activation = softmax, patience = 8, query = None, sort_on = Eta, useObjTypeColumn = True
--------------------------------------------------
-------

In [ ]:
trials = findWithMetrics(trials, {"useObjTypeColumn" : True})
print_by_labels(trials, 4)

#plot_history([("Fully connected:", trials[0].get_history())], plotLoss=False)


In [ ]:
t = findWithMetrics(trials, {"labels" : [u'ttbar', u'wjet', u'qcd'], 'sort_on' : "Phi"})
t[0].summary(showCompilation=True, showFit=True)

In [ ]:
plotEverything(trials)
    
    


In [ ]:
phi = findWithMetrics(trials,{"sort_on" : "Phi"})
eta = findWithMetrics(trials,{"sort_on" : "Eta"})
PT = findWithMetrics(trials,{"sort_on" : "PT_ET"})
sortOnMetric(phi, "labels")
sortOnMetric(eta, "labels")
sortOnMetric(PT, "labels")
labelGroups = zip(phi,eta,PT)

In [ ]:
colors = [(0,0,1.0),(.25,.75,.25), (1,.65,0)]
names = ["Phi", "Eta","PT"]
lims = [[.7,.91], [.8,1.0],  [.8,1.0],  [.4,.82]]
for j , tup in enumerate(labelGroups):
    plots = []
    for i, b in enumerate(tup):
        labels = b.get_from_record("labels")
        if(labels == None): labels = b.get_from_record("lables")
        title = str(tuple([str(x) for x in labels])) if(labels != None) else "Cannot Find Labels"
        title = title + " Accuracy vs Epoch"
        name = names[i]
        model = b.get_model(custom_objects={"Slice":Slice, "Lorentz" : Lorentz})
        history = b.get_history()
        color = colors[i]
        plots.append((name, history, color))
    plot_history(plots, plotLoss=False, title=title, acclims=lims[j])
    
    

In [ ]:
data = np.zeros(( len(labelGroups[0]),len(labelGroups) ) ).tolist()
print(data)
columns = [None] * len(labelGroups)
rows = [" "+ n + " " for n in names]

for j , tup in enumerate(labelGroups):
    for i, b in enumerate(tup):
        labels = b.get_from_record("labels")
        columns[j] = str(tuple([str(x) for x in labels]))
        error = getError(b,custom_objects={"Slice":Slice, "Lorentz" : Lorentz}, ignoreAssert=True)
        d = "%.5f %s %.5f" % (b.get_from_record("val_acc"),unichr(177), error)
        data[i][j] = d
        
        
#print(data)
        

In [ ]:
plotTable(rows, columns, data,scale=2, title="Validation Accuracy for LSTM Trials by Classification and Sorting")

In [ ]:
def getTrialBins(trial, bins=20):
    trial.summary()
    d = accVsEventChar(trial, None, np.sum, "PT_ET", ["EFlowPhoton","EFlowNeutralHadron","EFlowTrack"],
                       bins=bins,custom_objects={"Slice": Slice, "Lorentz": Lorentz},equalBins=False)
    #plotBins(d,title='Accuracy vs Sum of PF Candidate PT', xlabel="PT GeV", ylabel='Accuracy', color=(0.553,0.188,0.38))
    return d
eta_bins = [getTrialBins(t,40) for t in eta]
phi_bins = [getTrialBins(t,40) for t in phi]

In [ ]:
colors = [(0,0,1.0),(.25,.75,.25), (1,0,0), (1,.65,0)]
lables = [trial.get_from_record("labels") for trial in eta]
plotBins(eta_bins,mode="scatter",title='Eta-LSTM: Accuracy vs Sum of PF Candidate PT',binLabels=lables, xlabel="PT GeV", ylabel='Accuracy',
         legendTitle="Classification",colors=colors, alpha=.2, ylim=(0.75, 1.025), xlim=(0,3000))

lables = [trial.get_from_record("labels") for trial in phi]
plotBins(phi_bins,mode="scatter",title='Phi-LSTM: Accuracy vs Sum of PF Candidate PT',binLabels=lables, xlabel="PT GeV", ylabel='Accuracy',
         legendTitle="Classification",legendBelow=True,colors=colors, alpha=.4, ylim=(0.75, 1.025), xlim=(0,3000))